In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   -------------------------------- ------- 81.9/101.7 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 9.2 MB/s eta 0:00:00


In [2]:
pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/297.6 kB ? eta -:--:--
   --------- ------------------------------ 71.7/297.6 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 297.6/297.6 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

C:\Users\Rate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Tokenizer and model setup
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-2.7B')
tokenizer.pad_token = tokenizer.eos_token

C:\Users\Rate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rate\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Rate\AppData\Local\Packages\Pytho

In [5]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = inputs
        self.outputs = outputs
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        
        input_encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
        output_encodings = tokenizer(output_text, padding="max_length", truncation=True, max_length=self.max_length)
        
        return {
            "input_ids": torch.tensor(input_encodings['input_ids']),
            "attention_mask": torch.tensor(input_encodings['attention_mask']),
            "labels": torch.tensor(output_encodings['input_ids'])
        }
    

# Load and preprocess data
df = pd.read_json('Psychology-10K.json')
inputs = df['input'].tolist()
outputs = df['output'].tolist()
dataset = TextDataset(tokenizer, inputs, outputs)

In [6]:

# Training arguments and setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2, 
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    gradient_accumulation_steps= 10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


C:\Users\Rate\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
# Train the model
trainer.train()

  0%|          | 0/984 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def predict_response(input_text, model, tokenizer):
    # Preprocess and tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        max_length=512,  # Ensure the length matches what was used during training
        truncation=True,
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move the tensors to the same device as the model
    encoded_input = {key: tensor.to(model.device) for key, tensor in encoded_input.items()}

    # Make prediction
    with torch.no_grad():
        output = model(**encoded_input)

    # Retrieve the probabilities (logits) and find the predicted class
    logits = output.logits
    predicted_class_index = logits.argmax(-1).item()

    # If you used LabelEncoder during preprocessing, you need to reverse that encoding:
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel lonely"
response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

NameError: name 'model' is not defined